# Combine CSV Data <a id='top'></a>

***

## Table of Contents

* [Introduction](#intro)
* [Import Required Libraries](#import)
* [Read the CSV Data into Pandas Dataframe](#load)
* [Check Data](#check)
* [Combine `outcome` and `event` Data](#join1)
* [Combine with `fighter` Data](#join2)
* [Save Data](#save)
* [Show the Distribution of Fighters' Nationality](#distr)


## Introduction <a id='intro'></a>

First, we need to download the data, which are in CSV format:

1. Download **outcome** csv [here](http://www.sharecsv.com/s/e593dd089bc5aaf6583b5187f96cb48e/outcome_interview.csv). The outcome csv contains the matchup data for various mixed martial arts fights - including the two fighters involved, the winner, and the event where it took place - all encoded in IDs.

2. Download **event** csv [here](http://www.sharecsv.com/s/f9b678bf4a60bafe3f0a4132313e194d/event_interview.csv). The event csv contains event data for various mixed martial arts events - most importantly the event name. The id corresponds to the event_id in the outcome csv.

3. Download **fighter** csv [here](http://www.sharecsv.com/s/4f1b611a0d9f96d7700adba8b387df21/fighter_interview.csv). The fighter csv contains fighter data for various mixed martial arts fighters - their names and their nationalities. The id corresponds to both fighter1_id and fighter2_id in the outcome csv.

Using these CSVs and id keys, we will combine the data, rename column names, and also get rid of many columns. Last, we will make a visualization showing the distribution of fighters from various nationalities. The exported csv of all the matchups (reminder: matchups are listed in the outcome csv) will have only the following columns:

| fighter1_name | fighter2_name | fighter1_nationality | fighter2_nationality | event_name |

**Notes**

* The deadline for completion is 11:30PM (23:30) Pacific Standard Time on Wednesday May 22 (will be Thursday for most of your time zones). I will review all entries on Thursday May 23 PST (GMT -8).

* The jupyter notebook and the exported CSV will be uploaded to GitHub.

## Import Required Libraries <a id='import'></a>

In [ ]:
# Import required libraries.
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

[Go to the top](#top)

## Read the CSV Data into Pandas Dataframe <a id='load'></a>

Read the `CSV` data into `Pandas` Dataframes.

In [ ]:
# Read outcome csv.
outcome = pd.read_csv('outcome_interview.csv')

# Show first rows.
outcome.head()

In [ ]:
# Remove the unwanted column.
outcome.drop('winner_id', axis=1, inplace=True)

In [ ]:
# Read event csv.
event = pd.read_csv('event_interview.csv')

# Show first rows.
event.head()

In [ ]:
# Remove the unwanted columns.
event.drop(event.columns[2:], axis=1, inplace=True)

In [ ]:
# Read fighter csv.
fighter = pd.read_csv('fighter_interview.csv')

# Show first rows.
fighter.head()

[Go to the top](#top)

## Check Data <a id='check'></a>

The dataframes will be examined for the quality of data. The types and shape of the data will be checked, as well as if there are any duplicate and null records.

In [ ]:
# Create a function to check the data.
def check_data(df, key):
    """
    A function to check the data.
    Args:
        df: Pandas dataframe
        key: str, column name of dataframe
    """
    print('Data types:\n{}'.format(df.dtypes))
    print('\nShape (rows, columns):\n{}'.format(df.shape))
    print('\nMissing values:\n{}'.format(df.isnull().sum()))
    print('\nNumber of duplicated records: {}'.format(df.duplicated(key).sum()))
    
# Check outcome.
check_data(outcome, 'id')

In [ ]:
# Check event.
check_data(event, 'id')

In [ ]:
# Check fighter.
check_data(fighter, 'id')

[Go to the top](#top)

## Combine `outcome` and `event` Data <a id='join1'></a>

First, we will merge with an `inner` join the `outcome` and the `event` dataframes on the `event_id` and `id` keys correspondingly.

In [ ]:
# Join the dfs.
df = pd.merge(outcome, event, left_on='event_id', right_on='id', how='inner', sort=False)

# Show first rows.
df.head(2)

Then, we will rename the columns of interest and remove the rest.

In [ ]:
# Remove the unwanted columns.
df.drop(df.columns[[3, 4]], axis=1, inplace=True)

# Rename columns.
df.rename(columns={'name':'event_name'}, inplace=True)

# Show first rows.
df.head(2)

[Go to the top](#top)

## Combine with `fighter` Data <a id='join2'></a>

Now, we will combine with an `inner` join the `df` and the `fighter` dataframes on the `fighter1_id`, `fighter2_id`, and `id` keys.

In [ ]:
# First split the output df into 2 dfs, each one for fighter 1 and 2.
df1 = df.drop(['fighter2_id'], axis=1)
df2 = df.drop(['fighter1_id'], axis=1)

# Join the dfs.
df1 = pd.merge(df1, fighter, left_on='fighter1_id', right_on='id', how='inner')
df2 = pd.merge(df2, fighter, left_on='fighter2_id', right_on='id', how='inner')

# Rename columns.
df1.rename(columns={'name':'fighter1_name', 'nationality':'fighter1_nationality'}, inplace=True)
df2.rename(columns={'name':'fighter2_name', 'nationality':'fighter2_nationality'}, inplace=True)

# Drop columns.
df1.drop(df1.columns[[1, 2, 3]], axis=1, inplace=True)
df2.drop(df2.columns[[1, 3]], axis=1, inplace=True)

# Check data.
check_data(df1, 'id_x')
print(70*'-')
check_data(df2, 'id_x')

# Show first rows.
df1.head(2)

In [ ]:
# Show first rows.
df2.head(2)

In [ ]:
# Join the 'df1' and 'df2' on id_x.
final = pd.merge(df1, df2, on='id_x', how='inner')

# Re-arrange the order of the columns.
cols = ['id_x', 'fighter1_name', 'fighter2_name', 'fighter1_nationality', 'fighter2_nationality', 'event_name']

# Create a new dataframe with the columns in the desired order.
final = final[cols]

# Drop unwanted column.
final.drop('id_x', axis=1, inplace=True)

# Show first rows.
final.head()

[Go to the top](#top)

## Save Data <a id='save'></a>

Before we save the data we should check them.

In [ ]:
check_data(final, 'fighter1_name')

There are some missing nationality values. We can save a full final csv file and another one without the rows containing the missing values.

In [ ]:
# Export to csv.
final.to_csv('final.csv', index=False)

# Remove the rows with missing values and do a check.
final_without_nans = final.dropna()
check_data(final_without_nans, 'fighter1_name')

# Export to csv without the df index.
final_without_nans.to_csv('final_without_nans.csv', index=False)

[Go to the top](#top)

## Show the Distribution of Fighters' Nationality <a id='distr'></a>

To visualize the distribution of fighters from various nationalities, we need first to concatenate `fighter1_nationality` and `fighter2_nationality` and then aggregate the data with the `groupby` function.

In [ ]:
# Get the columns of nationality as Pandas series.
fighter1_nationality = final.fighter1_nationality
fighter2_nationality = final.fighter2_nationality

# Combine the two Series.
nationality = pd.concat([fighter1_nationality, fighter1_nationality], ignore_index=True)

# Convert to df.
nationality = pd.DataFrame({'Nationality':nationality.values})

# Show first rows.
nationality.head()

Create the chart.

In [ ]:
# Compute the unique values of the nationalities column and then create a bar plot.
ax = nationality['Nationality'].value_counts().plot(kind='barh', figsize=(20, 30))

# Annotate bars with values.
# Found from https://stackoverflow.com/questions/25447700/annotate-bars-with-values-on-pandas-bar-plots
for p in ax.patches:
    ax.annotate(str(p.get_width()), (p.get_x() + p.get_width(), p.get_y()),
                xytext=(10, 1), textcoords='offset points')

[Go to the top](#top)